# Pre-eminence tutorial

In this tutorial, you'll learn how to define an agent to play the game Pre-eminence, run games so you can track performance, and step through those games so you can debug & understand what your agent is doing at each point.

## 1. Creating an agent

In [ ]:
%load_ext autoreload
%autoreload 2
import preem
import random

In [ ]:
class RandomAgent(preem.Agent):
    def place(self, state):
        return random.choice(state.my_territories)

    def reinforce(self, state, count):
        return {random.choice(state.my_territories): count}

    def redeem(self, state):
        if 5 <= len(state.cards):
            sets = list(preem.get_matching_sets(state.cards))
            return random.choice(sets)

    def act(self, state, earned_card):
        territory = max(state.my_territories, key=lambda t: state.world.armies[t])
        armies = state.world.armies[territory]
        if 3 <= armies:
            enemy_neighbours = [t for t in state.map.edges[territory]
                                if state.world.owners[t] != state.player_index]
            if enemy_neighbours:
                return preem.Attack(from_=territory, to=random.choice(enemy_neighbours), count=armies - 1)
            return preem.Move(from_=territory,
                              to=random.choice(list(state.map.edges[territory])),
                              count=armies - 1)

In [ ]:
random.seed(10)
preem.Game.play(preem.Map.load_file('maps/mini.json'), [RandomAgent(), RandomAgent()])

In [ ]:
random.seed(10)
for event in preem.Game.start(preem.Map.load_file('maps/mini.json'), [RandomAgent(), RandomAgent()]):
    print(id(event.agent), event.method, event.args, '->', event.result)

## 2. Looking at the world

In [ ]:
random.seed(10)
game = preem.Game.start(preem.Map.load_file('maps/mini.json'), [RandomAgent(), RandomAgent()])
print(game.world)
game.map  # same as game.world.map

In [ ]:
event = next(game)
print(event)
print(event.state)
print(event.state.world)
print('territory owners', event.state.world.owners)
event

In [ ]:
while event.method != 'act':
    event = next(game)
print(event.result)
event

## 3. Running a mini-tournament

In [ ]:
# TODO